### COMP3314 ML
Guo Shunhua 3035447635 
<br>
# <u>Logistic regression Construction</u>

## Overview
1. [Introduction](#s1) 
2. [Decision Tree](#s2)
3. [Random Forest](#s3)
4. [Load data & Apply the model](#s4) 
5. [Parameter Analysis](#s5)


----- 


<a id=’s1’></a>

## 1 Introduction

This notebook will implement a Multi-class Decision Tree, Random Forest, and then train implemented model to provided data set.

<u>Import Libraries</u>

In [1]:
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected")
                    
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import statistics
import random

<a id=’s2’></a>

## 2 Decision Tree

As stated in the lecture, the following impurity measures or splitting criteria are commonly used in decision trees:
1. Gini impurity ( gini ), 
2. Entropy ( entropy ), 
3. Classification error ( classificationError ).


In [2]:
# define gini value for one group (classes)
def gini(group, numClass):
    gini = 1  
    size = len(group)
    
    if size != 0:
        y_v = [y[-1] for y in group]
        for c in range(numClass):
            #proportion of each class
            p = y_v.count(c)/size
            gini -= p**2
    return gini

print(gini([[1, 1], [1, 0]], 2), gini([[1, 0], [1, 0]], 2))

0.5 0.0


In [3]:
# define entropy value for one group (classes)
def entropy(group, numClass):
    entropy = 0  
    size = len(group)
    
    if size != 0:
        y_v = [y[-1] for y in group]
        for c in range(numClass):
            #proportion of each class
            p = y_v.count(c)/size
            if p != 0:
                entropy -= p * math.log(p,2)
        return entropy
    else:
        return 1
print(entropy([[1, 1], [1, 0]], 2), gini([[1, 0], [1, 0]], 2))

1.0 0.0


In [4]:
# define classificationError value for one group (classes)
def classificationError(group, numClass):
    p = []
    size = len(group)
    
    if size != 0:
        y_v = [y[-1] for y in group]
        for c in range(numClass):
            #proportion of each class
            p.append(y_v.count(c)/size)
        return 1 - np.max(p)     
    else:
        return 1
print(classificationError([[1, 1], [1, 0]], 2), gini([[1, 0], [1, 0]], 2)) 

0.5 0.0


#### Define a split function

In [5]:
# construct a split
def split(group, x_index, val):
    #record group spliting result
    left = []
    right = []
    for x in group:
        if x[x_index] < val:
            left.append(x)
        else:
            right.append(x)
    return left, right

#### Define a tree structure to store the Decision Tree training result

In [6]:
class DTnode(object):
    def __init__(self, level=0):
        self.level = level
        
        self.x_index = None
        self.val = None
        self.left = None
        self.right = None
        
    def update_info(self, x_index, val, left, right):
        self.x_index = x_index
        self.val = val
        self.left = left
        self.right = right
        
    def add_class(self, c):
        self.c = c
        
    def get_info(self):
        return self.x_index, self.val, self.left, self.right
    

In [7]:
class DecisionTree(object):
    def __init__(self, criterion='gini', max_depth=4, min_size=1):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_size = min_size
    
    # ------------------- Main ------------------- #

    def fit(self, X, y):
        self.numClass = int(np.max(y)) + 1
        X_w_y = np.concatenate((X, y), axis = 1)
        
        # gives a group split with max_depth
        # record each node spliting
        self.root = DTnode()
        self.split_node(self.root, X_w_y, self.max_depth)
        
        return self
    
    #recursively split the group
    def split_node(self, node, group, max_depth):
        if len(group) == 0:
            return
        
        if max_depth <= 0 or len(group) <= self.min_size:
            self.assign_class(node, group)
            return 
        
        if self.select_split(group) != None:
            (left, right), x_index, val = self.select_split(group)
            leftnode = DTnode(level=node.level+1)
            rightnode = DTnode(level=node.level+1)
            node.update_info(x_index, val, leftnode, rightnode)

            self.split_node(leftnode, left, max_depth-1)
            self.split_node(rightnode, right, max_depth-1)
        else:
            self.assign_class(node, group)
            return
    
    # record class classification at the node
    def assign_class(self, node, group):
        y_v = [y[-1] for y in group]
        c_v = [y_v.count(c) for c in range(self.numClass)]
        c = np.argmax(c_v)
        node.add_class(c)
        return
            
    # select split
    def select_split(self, group):
        info_gain_v = []
        info_gain_comb = []
        for x_index in range(len(group[0])-1):
            if len(group) > 100:   
                x_v = [x[x_index] for x in group]
                xmin, xmax = np.min(x_v), np.max(x_v)
                step_size = (xmax - xmin)/20
                val = xmin + step_size
                while val < xmax:
                    ig = self.info_gain(group, x_index, val)
                    info_gain_v.append(ig)
                    info_gain_comb.append([x_index, val])
                    val += step_size

            else:
                for x in group:
                    ig = self.info_gain(group, x_index, x[x_index])
                    info_gain_v.append(ig)
                    info_gain_comb.append([x_index, x[x_index]])

        # get the split resulting maximum infomation gain
        if np.max(info_gain_v) == 0:
            # information gain == 0, then no need to split
            return None
        else:
            x_index, val = info_gain_comb[np.argmax(info_gain_v)]
            return split(group, x_index, val), x_index, val    

    # compute information gain at a split point
    def info_gain(self, group, x_index, val):
        size = len(group)
        
        #parent's loss
        p = self.loss(group)

        left, right = split(group, x_index, val)

        # proportional left/right group's loss
        l = self.loss(left) * len(left) / size
        r = self.loss(right) * len(right) / size

        return p - l - r
    
    #define the impurity/error function called loss
    def loss(self, group):
        if self.criterion == "gini":
            return gini(group, self.numClass)
        elif self.criterion == "entropy":
            return entropy(group, self.numClass)
        elif self.criterion == "ce":
            return classificationError(group, self.numClass)
        else:
            raise Exception("Sorry, no this impurity/error function implemented.")

    # ------------------- Evalution ------------------- #
    
    # predict single x input
    def predict(self, node, x):
        x_index, val, leftnode, rightnode = node.get_info()
        
        if x_index == None or val == None:
            return node.c
        elif x[x_index] < val:
            return self.predict(leftnode, x)
        else:
            return self.predict(rightnode, x)
    
    #compute model accuracy
    def accuracy(self, X, y):
        y_pred = [[self.predict(self.root, x)] for x in X]
        return (y_pred == y).sum()/len(y)

<a id=’s3’></a>


## 3 Random Forest

In [8]:
class RandomForest(object):
    def __init__(self, criterion='gini', max_depth=4, min_size=1, 
                 k_estimators=25, random_seed=42):
        
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_size = min_size
        self.k_estimators = k_estimators
        self.random_seed = random_seed
        
    # ------------------- Main ------------------- #

    def fit(self, X, y, n_sample_size=None, d_features=None):
        
        # define n sample size if not yet defined
        if n_sample_size == None:
            print("Reminder: n_sample_size is not specified here, assume to be total/5")
            self.n_sample_size = int(len(y)/5)
        else:
            self.n_sample_size = n_sample_size
        
        # define d feature number if not yet defined
        if d_features == None:
            print("Reminder: d_features is not specified here, assume to be the whole feature set")
            self.d_features = len(X[0])
        else:
            self.d_features = d_features
        
        self.trees = []
        for i in range(self.k_estimators):
            #set random seed
            random.seed(int(self.random_seed))
            self.random_seed += 1
            
            rnd_index = random.sample(range(len(y)), self.n_sample_size)
            feature_index = set(random.sample(range(len(X[0])), self.d_features))
            
            # masking non-seleted columns (set to all zeros)
            X_sampled = X[rnd_index]
            X_sampled = [X_sampled[:,i] if i in feature_index else X_sampled[:,i]*0 for i in range(len(X[0]))]
            X_sampled = np.stack(X_sampled, axis = 1)
            
            y_sampled = y[rnd_index]
            
            self.trees.append(self.return_DT(X_sampled, y_sampled))

        return self
    
    def return_DT(self, X_sampled, y_sampled):
        tree = DecisionTree(criterion=self.criterion, 
                            max_depth=self.max_depth, min_size=self.min_size)
        return tree.fit(X_sampled, y_sampled)
    
    # ------------------- Evalution ------------------- #
    
    #predict single x sample
    def predict(self, x):
        y_preds = [tree.predict(tree.root, x) for tree in self.trees]
        return statistics.mode(y_preds)
    
    # calculate accuracy of the model
    def accuracy(self, X, y):
        y_pred = [[self.predict(x)] for x in X]
        return (y_pred == y).sum()/len(y)

<a id=’s4’></a>

## 4 Load data & Apply the model

### Iris data set

In [9]:
# Load data from csv file, which located in the same folder as this notebook.

X_train = pd.read_csv("dataset_files/iris_X_train.csv").to_numpy()
y_train = pd.read_csv("dataset_files/iris_y_train.csv").to_numpy()
X_test = pd.read_csv("dataset_files/iris_X_test.csv").to_numpy()
y_test = pd.read_csv("dataset_files/iris_y_test.csv").to_numpy()

#### Single Decision Tree training result

In [10]:
tree = DecisionTree()
tree.fit(X_train, y_train)

print("training accuracy: ", tree.accuracy(X_train, y_train))
print("test accuracy: ", tree.accuracy(X_test, y_test))

training accuracy:  0.98
test accuracy:  0.98


#### Random Forest training result

In [11]:
forest = RandomForest(k_estimators=100)
forest.fit(X_train, y_train, d_features=5)

print("training accuracy: ", forest.accuracy(X_train, y_train))
print("test accuracy: ", forest.accuracy(X_test, y_test))

Reminder: n_sample_size is not specified here, assume to be total/5


ValueError: Sample larger than population or is negative

### Car data set

In [ ]:
# Load raw data from csv file, which located in the same folder as this notebook.

X_train_raw = pd.read_csv("dataset_files/car_X_train.csv").to_numpy()
y_train_raw = pd.read_csv("dataset_files/car_y_train.csv").to_numpy()
X_test_raw = pd.read_csv("dataset_files/car_X_test.csv").to_numpy()
y_test_raw = pd.read_csv("dataset_files/car_y_test.csv").to_numpy()

print("X input Meaning: ",
      "buying price, price of the maintenance, number of doors, ",
      "number of persons to carry, size of luggage boot, safety of the car")

print("X training Class List: ", set(X_train_raw.flatten()))
print("y training Class List: ", set(y_train_raw.flatten()))
print("X test Class List: ", set(X_test_raw.flatten()))
print("y test Class List: ", set(y_test_raw.flatten()))

In [ ]:
# transform str indicators in data to numeric classes
x_dict = {
    'vhigh' : 4, 
    'high' : 3,
    'med' : 2,
    'low' : 1, 
     
    'small' : 1,
    # 'med' : 2,
    'big' : 3,
    
    # '2' : 2, 
    # '4' : 4,
    'more' : 6, 
    
    '2' : 2, 
    '3' : 3,
    '4' : 4,
    '5more' : 6,
}

y_dict = {
    'acc':1, 'good':2, 'unacc':0, 'vgood':3
}

def transform_X(X):
    for i in range(len(X)):
        x = X[i]
        X[i] = [x_dict[i] for i in x]
    return X

def transform_y(y):
    return [[y_dict[row[0]]] for row in y]

X_train = transform_X(X_train_raw)
y_train = np.array(transform_y(y_train_raw))
X_test = transform_X(X_test_raw)
y_test = np.array(transform_y(y_test_raw))

#### Single Decision Tree training result

In [ ]:
tree = DecisionTree()
tree.fit(X_train, y_train)

print("training accuracy: ", tree.accuracy(X_train, y_train))
print("test accuracy: ", tree.accuracy(X_test, y_test))

#### Random Forest training result

In [ ]:
forest = RandomForest(k_estimators=100)
forest.fit(X_train, y_train, d_features=5)

print("training accuracy: ", forest.accuracy(X_train, y_train))
print("test accuracy: ", forest.accuracy(X_test, y_test))

<a id=’s5’></a>

## 5 Parameter Analysis

The data set used in this section is Car Data Set.

#### Criteria

In [ ]:
# record accuracy in each case
accu = []
# Gini
tree = DecisionTree()
tree.fit(X_train, y_train)
forest = RandomForest()
forest.fit(X_train, y_train, n_sample_size=100, d_features=5)

accu.append([tree.accuracy(X_train, y_train), tree.accuracy(X_test, y_test),
            forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])

# Entropy
tree = DecisionTree(criterion="entropy")
tree.fit(X_train, y_train)
forest = RandomForest(criterion="entropy")
forest.fit(X_train, y_train, n_sample_size=100, d_features=5)

accu.append([tree.accuracy(X_train, y_train), tree.accuracy(X_test, y_test),
            forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])

# ClassificationError
tree = DecisionTree(criterion="ce")
tree.fit(X_train, y_train)
forest = RandomForest(criterion="ce")
forest.fit(X_train, y_train, n_sample_size=100, d_features=5)

accu.append([tree.accuracy(X_train, y_train), tree.accuracy(X_test, y_test),
            forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])
accu_pd = pd.DataFrame(accu, 
             columns=['DT training', 'DT test', 'RF training', 'RF test'],
             index = ["Gini", "Entropy", "ClassificationError"])

In [ ]:
import seaborn as sns

sns.heatmap(accu_pd, annot=True)

#### Max_depth

In [ ]:
x = range(1,20)

# record accuracy in each case
accu = []

for i in x:
    tree = DecisionTree(max_depth=i)
    tree.fit(X_train, y_train)
    forest = RandomForest(max_depth=i)
    forest.fit(X_train, y_train, n_sample_size=100, d_features=5)

    accu.append([tree.accuracy(X_train, y_train), tree.accuracy(X_test, y_test),
                forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])

accu = np.array(accu).T

plt.plot(x, accu[0], label="DT training")
plt.plot(x, accu[1], label="DT test")
plt.plot(x, accu[2], label="RF training")
plt.plot(x, accu[3], label="RF test")

plt.xlabel("max_dept")
plt.ylabel("Accuracy")
plt.legend(loc="best")
plt.show()

#### Number of tree estimators in Random Forest (k)

In [ ]:
x = range(10,200,5)

# record accuracy in each case
accu = []

for i in x:
    forest = RandomForest(k_estimators=i)
    forest.fit(X_train, y_train, n_sample_size=100, d_features=5)

    accu.append([forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])

accu = np.array(accu).T

plt.plot(x, accu[0], label="training")
plt.plot(x, accu[1], label="test")

plt.xlabel("Number of tree estimators in Random Forest")
plt.ylabel("RT Accuracy")
plt.legend(loc="best")
plt.show()

#### Number of sample size in each sample bagging (n)

In [ ]:
x = range(int(len(y_train)/20),len(y_train)+1,int(len(y_train)/20))

# record accuracy in each case
accu = []

for i in x:
    forest = RandomForest()
    forest.fit(X_train, y_train, n_sample_size=i, d_features=5)
    #print(forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test))
    accu.append([forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])

accu = np.array(accu).T

plt.plot(x, accu[0], label="training")
plt.plot(x, accu[1], label="test")

plt.xlabel("Number of tree estimators in Random Forest")
plt.ylabel("RT Accuracy")
plt.legend(loc="best")
plt.show()

#### Number of feature selected in each sample bagging (d)

In [ ]:
x = range(1, len(X_train[0]) + 1)

# record accuracy in each case
accu = []

for i in x:
    forest = RandomForest()
    forest.fit(X_train, y_train, n_sample_size=100, d_features=i)
    #print(forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test))
    accu.append([forest.accuracy(X_train, y_train), forest.accuracy(X_test, y_test)])

accu = np.array(accu).T

plt.plot(x, accu[0], label="training")
plt.plot(x, accu[1], label="test")

plt.xlabel("Number of tree estimators in Random Forest")
plt.ylabel("RT Accuracy")
plt.legend(loc="best")
plt.show()